# Examples #

This notebook demonstrates usage of IBM qiskit, how to deal in this repository with circuits to be executed on the simulator, and those to be properly run on the backend ('queue').

In [ ]:
'''IMPORTS'''
import os;
import sys;

# NOTE: need this to force jupyter to reload imports:
for key in list(sys.modules.keys()):
    if key.startswith('src.'):
        del sys.modules[key];

os.chdir(os.path.dirname(_dh[0]));
sys.path.insert(0, os.getcwd());

from src.thirdparty.maths import *;
from src.thirdparty.quantum import *;
from src.api.ibm import *;
from src.demo.examples import *;

np.random.seed(39102901); # for repeatability

In [ ]:
'''Example circuit'''
action_display_circuit(theta1=np.pi/3, theta2=0, theta3=-np.pi/4);

In [ ]:
'''Example usage of simulator'''
action_prepare_circuit_and_job(
    option = BACKEND_SIMULATOR.AER,
    num_shots = 10000,
    theta1 = np.pi/3,
    theta2 = 0,
    theta3 = -np.pi/4,
);

action_display_statistics(queue=False);

In [ ]:
'''Display list of IBM backend options'''
display_backends();

In [ ]:
'''Example usage of IBM cloud backend (queue)'''
action_prepare_circuit_and_job(
    option = BACKEND.LEAST_BUSY,
    num_shots = 100,
    theta1 = np.pi/3,
    theta2 = 0,
    theta3 = -np.pi/4,
);

In [ ]:
'''Statistics for backend job - NOTE: job may be pending'''
action_display_statistics(queue=True);